In [7]:
import pandas as pd
import numpy as np
import re
import math
from collections import Counter
import operator

In [14]:
movie_for_ml = pd.read_csv('data/movies_merged.csv.zip')

In [15]:
movie_for_ml.head()

,tconst,originalTitle,data,cluster
0,tt0008879,Berg-Ejvind och hans hustru,"1918,Drama",5
1,tt0009893,Die Austernprinzessin,"1919,Comedy",24
2,tt0009937,Blind Husbands,"1919,Drama,Romance",4
3,tt0009968,Broken Blossoms or The Yellow Man and the Girl,"1919,Drama,Romance",4
4,tt0010247,Herr Arnes pengar,"1919,Drama,History",18


In [16]:
movie_title = 'Herself'
cluster_number = movie_for_ml.loc[movie_for_ml['originalTitle']==movie_title,'cluster']
cluster_number

11652    5
Name: cluster, dtype: int64

In [22]:
movie_data = movie_for_ml.loc[movie_for_ml['originalTitle']==movie_title,'data']
movie_data

11652    2020,Drama
Name: data, dtype: object

In [18]:
class CosineSimilarity:
    def __init__(self):
        print("Cosine Similarity initialized")

    @staticmethod
    def cosine_similarity_of(text1, text2):
        #get words first
        first = re.compile(r"[\w']+").findall(text1)
        second = re.compile(r"[\w']+").findall(text2)

        #get dictionary with each word and count.
        vector1 = Counter(first)
        vector2 = Counter(second)

        #convert vectors to set to find common words as intersection
        common = set(vector1.keys()).intersection(set(vector2.keys()))

        dot_product = 0.0

        for i in common:
            #get amount of each common word for both vectors and multiply them then add them together.
            dot_product += vector1[i] * vector2[i]

        squared_sum_vector1 = 0.0
        squared_sum_vector2 = 0.0

        #get squared sum values of word counts from each vector.
        for i in vector1.keys():
            squared_sum_vector1 += vector1[i]**2

        for i in vector2.keys():
            squared_sum_vector2 += vector2[i]**2

        #calculate magnitude with squared sums.
        magnitude = math.sqrt(squared_sum_vector1) * math.sqrt(squared_sum_vector2)

        if not magnitude:
           return 0.0
        else:
           return float(dot_product) / magnitude

In [19]:
class RecommenderEngine:
    def __init__(self):
        print("engine initialized")

    def get_recommendations(keywords):

        df = movie_for_ml[movie_for_ml['cluster']==cluster_number]

        score_dict = {}

        for index, row in df.iterrows():
            score_dict[index] = CosineSimilarity.cosine_similarity_of(row['data'], keywords)

        #sort cities by score and index.
        sorted_scores = sorted(score_dict.items(), key=operator.itemgetter(1), reverse=True)

        counter = 0

        #create an empty results data frame.
        resultDF = pd.DataFrame(columns=('tconst', 'originalTitle', 'data', 'score'))

        #get highest scored 5 cities.
        for i in sorted_scores:
            #print index and score of the city.
            #print(i[0], i[1])
            resultDF = resultDF.append({'tconst': df.iloc[i[0]]['tconst'], 'originalTitle': df.iloc[i[0]]['originalTitle'], 'data': df.iloc[i[0]]['data'], 'score': i[1]}, ignore_index=True)
            counter += 1

            if counter>4:
                break
        return resultDF

In [21]:
def get_recommendations(keywords):
    return RecommenderEngine.get_recommendations(keywords)

In [23]:
get_recommendations(movie_data)

ValueError: Can only compare identically-labeled Series objects